In [ ]:
# Core
import sys
import pandas as pd
import numpy as np
from pathlib import Path

## Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

## Harp/Bonsai
sys.path.append('../../src/')
from bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
import harp
import harp.processing
import data_io



In [ ]:
#Global Viz settings
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

default_img_size = (15, 8)

In [ ]:
session_path = Path(r"Z:\scratch\vr-foraging\672102\20231012T094718")

session_path = Path(r"Z:\scratch\vr-foraging\672103\20231012T105405")
session_path = Path(r"Z:\scratch\vr-foraging\672104\20231013T092240")


# Harp Devices:
HarpBehavior = harp.HarpDevice("Behavior")
harp_behavior_data = data_io.HarpSource(device=HarpBehavior, path=session_path / "Behavior", name="behavior", autoload=False)

software_events = data_io.SoftwareEventSource(path=session_path / "SoftwareEvents", name="software_events", autoload=True)
config = data_io.ConfigSource(path=session_path / "Config", name="config", autoload=True)
operation_control = data_io.OperationControlSource(path=session_path / "OperationControl", name="config", autoload=False)


In [ ]:
## Find changes in reward available
patches = software_events.streams.ActivePatch.data
sites = software_events.streams.ActiveSite.data

reward_available_in_patch = software_events.streams.RewardAvailableInPatch.data
give_reward = software_events.streams.GiveReward.data
choice_feedback = software_events.streams.ChoiceFeedback.data

sites["patch_number"] = -1
sites["site_number"] = -1
sites["is_choice"] = False
sites["is_reward"] = 0
sites["past_no_reward_count"] = 0
sites["reward_available_in_patch"] = 0
past_no_reward_counter = 0
current_patch_idx = -1

site_number = 0
for idx, event in enumerate(sites.iterrows()):

    #patch_number
    arg_min, _ = harp.processing.find_closest(
        event[0],
        patches.index.values,
        mode="below_zero")
    if not (np.isnan(arg_min)):
        sites.loc[event[0], "patch_number"] = arg_min
    
    if event[1]["data"]["label"] == "Reward":
        if current_patch_idx != arg_min:
            current_patch_idx = arg_min
            site_number = 0
        else:
            site_number += 1
        sites.loc[event[0], "site_number"] = site_number
    else:
        sites.loc[event[0], "site_number"] = np.nan
    #available reward
    arg_min, _ = harp.processing.find_closest(
        event[0],
        reward_available_in_patch.index.values,
        mode="below_zero")
    if not (np.isnan(arg_min)):
        sites.loc[event[0], "reward_available_in_patch"] = reward_available_in_patch.iloc[arg_min]["data"]

    # outcomes
    if idx < len(sites) - 1:
        choice = choice_feedback.loc[(choice_feedback.index >= sites.index[idx]) & (choice_feedback.index < sites.index[idx+1])]
        is_reward = give_reward.loc[(give_reward.index >= sites.index[idx]) & (give_reward.index < sites.index[idx+1])]
    else: #account for the last trial
        choice = choice_feedback.loc[(choice_feedback.index >= sites.index[idx])]
        is_reward = give_reward.loc[(give_reward.index >= sites.index[idx])]

    sites.loc[event[0], "is_choice"] = len(choice) > 0
    sites.loc[event[0], "is_reward"] = is_reward.iloc[0]["data"] if len(is_reward) > 0 else np.nan
    sites.loc[event[0], "past_no_reward_count"] = past_no_reward_counter
    if sites.loc[event[0], "is_reward"] == 0:
        past_no_reward_counter += 1
    elif sites.loc[event[0], "is_reward"] > 0:
        past_no_reward_counter = 0

sites["patch_label"] = sites["patch_number"].apply(lambda x : patches.iloc[x]["data"]["label"])
sites["site_type"] = sites["data"].apply(lambda x: x["label"])


In [ ]:
# Odor onsets
## mask for digital outputs
digital_outputs = HarpBehavior.module.DigitalOutputs

## Load necessary files
harp_behavior_data.streams.OutputSet.load_from_file()
harp_behavior_data.streams.OutputClear.load_from_file()
odor_0 = harp.processing.distinct_until_changed_state(harp_behavior_data.streams.OutputSet.data, harp_behavior_data.streams.OutputClear.data, digital_outputs.SupplyPort1)
odor_1 = harp.processing.distinct_until_changed_state(harp_behavior_data.streams.OutputSet.data, harp_behavior_data.streams.OutputClear.data, digital_outputs.SupplyPort2)

odors_onset = pd.concat(
    [
        odor_0[odor_0["Value"]].assign(Value=0),
        odor_1[odor_1["Value"]].assign(Value=1),
    ], axis=0, copy=True).sort_index()


treadmill_metadata = config.streams.Rig.data["treadmill"]
encoder = harp.read_harp_bin(harp_behavior_data.streams.AnalogData.path).iloc[:,1]
converter = treadmill_metadata["wheelDiameter"] * np.pi / treadmill_metadata["pulsesPerRevolution"] * (-1 if treadmill_metadata["invertDirection"] else 1)
encoder = encoder.apply(lambda x : x * converter)
encoder.index = pd.to_datetime(encoder.index, unit="s")
encoder = encoder.resample("33ms").sum().interpolate(method="linear") / 0.033
encoder.index = (encoder.index - pd.to_datetime(0))
encoder.index = encoder.index.total_seconds()

In [ ]:

session_duration = sites.index[-1] - sites.index[0]
threshold = sites.index[0] +  session_duration * 0.85


window = (-0.2, 1)
fig, axs = plt.subplots(3, 2, sharex=True, figsize=(9, 10))
sp_index = 0
colors = [
    '#d73027',
    '#fdae61',
    '#abd9e9',
    '#4575b4'
    ]


y_axis = {'Reward' : 0, 'Gap': 1, 'InterPatch': 2}
x_axis = {'Bananas' : 0, 'PineBerries': 1, 'Ethyl Butyrate': 1, 'Octanol': 0}


for site_label, site_df in sites[sites.index < threshold].groupby(["site_type", "patch_label"]):
    for rwd_available_in_patch, rwd_avail_df in site_df.groupby('reward_available_in_patch'):
        trials = []
        for trial in rwd_avail_df.iterrows():
            t = trial[0]
            enconder_slice = encoder.loc[t + window[0]: t + window[1]]
            axs[y_axis[site_label[0]], x_axis[site_label[1]]].plot(
                enconder_slice.index.values - t,
                enconder_slice.values,
                color=colors[trial[1].reward_available_in_patch], alpha=0.1, lw = 1)
            trials.append(enconder_slice.values)
        min_len = min([len(trial) for trial in trials])
        trials = [trial[:min_len] for trial in trials]
        axs[y_axis[site_label[0]], x_axis[site_label[1]]].plot(
                        np.linspace(window[0], window[1], min_len),
                        np.mean(np.array(trials), axis=0),
                        color=colors[trial[1].reward_available_in_patch], alpha=.8, lw = 3)
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].vlines(0, -10, 60, color="k", linestyle="--")
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_title(site_label)
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_xlabel("Time(s)")
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_ylabel("Velocity (cm/s)")
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_ylim((-10, 60))
    sp_index += 1
plt.figure()
[plt.plot((0,0), (0,0), lw=3, c=colors[i], label=f"{i}") for i in range(4)]
plt.legend()
plt.show()

In [ ]:

session_duration = sites.index[-1] - sites.index[0]
threshold = sites.index[0] +  session_duration * 0.85


window = (-0.2, 1)
fig, axs = plt.subplots(2, 2, sharex=True, figsize=(9, 10))
sp_index = 0
colors = [
    '#d73027',
    '#fdae61',
    '#abd9e9',
    '#4575b4'
    ]


y_axis = {True : 0, 'Gap': 1, False: 1}
x_axis = {'Bananas' : 0, 'PineBerries': 1, 'Ethyl Butyrate': 1, 'Octanol': 0}


for site_label, site_df in sites[(sites.index < threshold) & (sites.site_type == 'Reward')].groupby(["is_choice", "patch_label"]):
    for rwd_available_in_patch, rwd_avail_df in site_df.groupby('reward_available_in_patch'):
        trials = []
        for trial in rwd_avail_df.iterrows():
            t = trial[0]
            enconder_slice = encoder.loc[t + window[0]: t + window[1]]
            axs[y_axis[site_label[0]], x_axis[site_label[1]]].plot(
                enconder_slice.index.values - t,
                enconder_slice.values,
                color=colors[trial[1].reward_available_in_patch], alpha=0.1, lw = 1)
            trials.append(enconder_slice.values)
        min_len = min([len(trial) for trial in trials])
        trials = [trial[:min_len] for trial in trials]
        axs[y_axis[site_label[0]], x_axis[site_label[1]]].plot(
                        np.linspace(window[0], window[1], min_len),
                        np.mean(np.array(trials), axis=0),
                        color=colors[trial[1].reward_available_in_patch], alpha=.8, lw = 3)
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].vlines(0, -10, 60, color="k", linestyle="--")
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_title(site_label)
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_xlabel("Time(s)")
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_ylabel("Velocity (cm/s)")
    axs[y_axis[site_label[0]], x_axis[site_label[1]]].set_ylim((-10, 60))
    sp_index += 1
plt.figure()
[plt.plot((0,0), (0,0), lw=3, c=colors[i], label=f"{i}") for i in range(4)]
plt.legend()
plt.show()